# U.S. Stock Market V: Shares Outstanding

In [ ]:
import pandas as pd
import numpy as np
import requests, zipfile, io
import os
from pathlib import Path

from tiingo import TiingoClient                       
tiingo = TiingoClient({'api_key':'XXXX'})

import matplotlib.pyplot as plt                        # Basic plot library.
plt.style.use('ggplot')                                # Make plots look nice.

Get shares:

In [ ]:
# Example:
directory = 'data/sec/merged/'
filename  = '2020q1.csv'
data = pd.read_csv(directory+filename, parse_dates=['filed','ddate'])

tag  = 'EntityCommonStockSharesOutstanding'
item = data[data.tag==tag] 

sort = item.sort_values(['cik','filed','ddate'], ascending=[True,True,True])  [:3]
sort

For each firm and each filing date and each ddate, sum all shares and get value column (possibly multiplt share classes):

In [ ]:
sort.groupby(['cik','filed','ddate']).sum()[['value']]

Put this into a function:

In [ ]:
def get_shares_from_SEC_files(tags, filename=None):               # Function input: list of tags, optional filename.

    directory = 'data/sec/merged/'                                # Read data from here.
    filenames = [filename] if filename else os.listdir(directory) # Supplied filename or all files in "merged" directory.
    filenames = [f for f in filenames if not f.startswith(".")]   # Exclude hidden files from file list.


    results   = {t:pd.DataFrame() for t in tags}                  # Dictionary of tables (1 table for each tag)

    for filename in filenames:                                    # Loop over all files.
        print(filename)
        data = pd.read_csv(directory+filename, parse_dates=['filed','ddate'])  # Read the file.
        
        for t in tags:                                            # Loop over all tags.
            item   = data[data.tag==t]                             # Select all data for this tag.
            sort   = item.sort_values(['cik','filed','ddate'], ascending=[True,True,True])
            total  = sort.groupby(['cik','filed','ddate']).sum()
            latest = total.groupby(['cik','filed']).last()
            results[t] = results[t].append( latest )  
                        
    for t in tags:                                                # Now sort all tables by filing date.
        if not results[t].empty: results[t] = results[t].sort_index(level='filed')[['value']]            

    return results

In [ ]:
tags = ['EntityCommonStockSharesOutstanding','CommonStockSharesOutstanding']

items = get_shares_from_SEC_files(tags)

EntityCommonStockSharesOutstanding:

Set zero shares to missing:

In [ ]:
items['EntityCommonStockSharesOutstanding'] = items['EntityCommonStockSharesOutstanding'].replace(0, np.nan) 
items['CommonStockSharesOutstanding']       = items['CommonStockSharesOutstanding']      .replace(0, np.nan)  

Combine these two tags (replace missing EntityCommonStockSharesOutstanding with CommonStockSharesOutstanding):

In [ ]:
def combine_items(tags, items):
    result = items[tags[0]]
    for tag in tags[1:]:  result = result.combine_first( items[tag] )
    return result


shares = combine_items(tags, items)
shares

In [ ]:
# Save this table:
shares.to_csv('data/sec/items/SharesOutstanding.csv')

In [ ]:
# Read the table we just saved:
shares = pd.read_csv('data/sec/items/SharesOutstanding.csv', parse_dates=['filed'], index_col=['cik','filed'])
shares

Forward-fill shares to all trading days:

In [ ]:
def ffill_values(item, dates):                                          
    data = item.unstack('cik')
    data = data.reindex(dates.union(data.index)).sort_index()           # Add specified dates to index.
    filing_dates = pd.read_csv('data/sec/dates/filing_dates.csv', index_col='cik', parse_dates=['filed']).filed
    last_filing_date_all_firms = filing_dates.max()                     # Most recent date where at least 1 firm filed.
     
    for cik in data.columns:                                            # Loop over all firms.
        last_filing_date      = pd.Series(filing_dates[cik]).iloc[-1]   # Last date where this firm filed
        days_since_last_filed = (last_filing_date_all_firms - last_filing_date).days
        last_date_this_firm   = dates[-1] if days_since_last_filed < 120 else last_filing_date
        data.loc[:last_date_this_firm, cik].ffill(inplace=True)         # Forward fill all the values.

    return data.loc[dates]                                              # Return only specified dates.   



trading_days = pd.to_datetime( tiingo.get_dataframe('SPY','2009-04-15').index ).tz_convert(None)

shares = ffill_values(shares.value, trading_days)

shares[-3:]

Shares outstanding for specific firm:

In [ ]:
symbols = pd.read_csv('data/ticker_symbols/symbols.csv',index_col=0)

In [ ]:
cik = symbols[symbols.ticker==''].index[0]  



Get some prices:

In [ ]:
close = tiingo.get_dataframe(['AAPL','MSFT','AMZN','TSLA'], '2009-01-01', metric_name='close')
close.index = pd.to_datetime(close.index).tz_convert(None)
close

Plot Apple close price:

Calculate market cap for single firm:

In [ ]:
shares = shares.rename( columns=symbols.ticker )
shares[-3:]

In [ ]:
ticker = 'AAPL'



Compare shares and price of Apple from 2020-5 to 2020-11:

In [ ]:
pd.DataFrame({'Shares':shares[ticker]/10**8, 'Price':close[ticker]})

Get stock splits for these firms:

In [ ]:
splitFactor = tiingo.get_dataframe(['AAPL','MSFT','AMZN','TSLA'], '2009-01-01', metric_name='splitFactor')
splitFactor.index = pd.to_datetime(splitFactor.index).tz_convert(None)
splitFactor

Generate table with split adjustments:

In [ ]:
filing_dates = pd.read_csv('data/sec/dates/filing_dates.csv', index_col='cik', parse_dates=['filed']).filed

ones = pd.DataFrame({'date':filing_dates, 'one':1})
ones = ones.set_index([ones.index,ones.date]).one.unstack('cik')
ones = ones.rename( columns=symbols.ticker )
ones = ones.reindex( trading_days.union(ones.index) )

split_adjustments = ones.fillna( splitFactor[splitFactor!=1] )
split_adjustments = split_adjustments.ffill()

split_adjustments

Plot these values for specific firm:

Market cap for this firm:

In [ ]:
ticker = ''



Caculate market cap for all firms:

In [ ]:
mcap = 
mcap

Check market cap of specific firm:

Get sales and earnings:

In [ ]:
trading_days = tiingo.get_dataframe('SPY','2009-04-15').index.tz_convert(None)

sales    = pd.read_csv('data/sec/items/Sales.csv',    parse_dates=['filed'], index_col=['cik','filed'])
earnings = pd.read_csv('data/sec/items/Earnings.csv', parse_dates=['filed'], index_col=['cik','filed'])

salesQ = ffill_values(sales.valueQ, trading_days)
salesA = ffill_values(sales.valueA, trading_days)

earningsQ = ffill_values( earnings.valueQ, trading_days )  
earningsA = ffill_values( earnings.valueA, trading_days )

Price-earnings ratio

In [ ]:
ticker = ''

cik = symbols[symbols.ticker==ticker].index[0]  

t = pd.DataFrame()
t['PE quarterly'] =
t['PE annual']    = 

t